In [3]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import sys

In [4]:
image1 = Image.open("paris/paris_a.jpg")
image2 = Image.open("paris/paris_b.jpg")
img1 = np.asarray(image1)
img2 = np.asarray(image2)
#plt.imshow(image)
#points = plt.ginput(5, show_clicks=True)

In [5]:
np.shape(img1)

(480, 640, 3)

In [86]:
with open('paris1-2_15points.npy', 'rb') as f:
    a = np.load(f)
    b = np.load(f)

In [75]:
a

array([[328.07230856, 218.52507106],
       [310.52135476, 246.0545867 ],
       [334.76594613, 318.44014258],
       [445.64545901, 326.37323018],
       [437.88943571, 271.75345296]])

In [76]:
b

array([[ 41.55208071, 209.79628183],
       [ 15.2006131 , 239.52368902],
       [ 43.12193581, 324.2018326 ],
       [180.37196077, 328.55792604],
       [165.44159128, 273.10212136]])

In [60]:
def computeH(points_im1, points_im2):
    matrixA = []
    for i in range(len(points_im1)):
        xs, ys = points_im1[i][0], points_im1[i][1]
        xd, yd = points_im2[i][0], points_im1[i][1]
        two_nineMatrix = np.array([[xs, ys, 1,0,0,0, -xd*xs, -xd*ys, -xd],
                                   [0,0,0,xs,ys,1,-yd*xs,-yd*ys,-yd]])
        matrixA.append(two_nineMatrix[0])
        matrixA.append(two_nineMatrix[1])
    return matrixA

In [268]:
A = computeH(b,a)

In [269]:
u, s, vh = np.linalg.svd(A, full_matrices=True)

In [270]:
print(s)
homography = vh[-1].reshape(3,3)
print(homography)
'''
homography = np.array(
    [[ 2.06417031e+00, -1.09152156e-01, -5.91002794e+02],
 [ 4.98137666e-01,  1.73071462e+00, -2.18682112e+02],
 [ 1.70508897e-03, -9.86837491e-05,  1.00000000e+00]])
 '''

[8.81059134e+05 1.57433460e+05 1.41289863e+03 3.31414130e+02
 2.56984118e+02 5.70454640e+01 3.85351720e+01 2.41428994e-01
 5.25026449e-02]
[[ 1.56741990e-03 -4.45676666e-04  9.85287655e-01]
 [-6.25258809e-04  2.27373072e-03  1.70850202e-01]
 [-2.05876698e-06 -1.10615918e-06  3.19838151e-03]]


'\nhomography = np.array(\n    [[ 2.06417031e+00, -1.09152156e-01, -5.91002794e+02],\n [ 4.98137666e-01,  1.73071462e+00, -2.18682112e+02],\n [ 1.70508897e-03, -9.86837491e-05,  1.00000000e+00]])\n '

In [54]:
def interpolation(image, x,y):
    leftTopX, leftTopY, leftTopVal = int(x), int(y), image[int(x)][int(y)]
    rightTopX, rightTopY, rightTopVal = int(x) +1 , int(y), image[int(x)+1][int(y)]
    leftBottomX, leftBottomY, leftBottomVal = int(x), int(y)+1, image[int(x)][int(y)+1]
    rightBottomX, rightBottomY, rightBottomVal = int(x)+1, int(y)+1, image[int(x)+1][int(y)+1]
    a=x%1
    b= int(y)+1 - y
    return leftBottomVal * (1- a)*(1-b) + leftTopVal * (1-a) * b + rightBottomVal * a*(1-b) + rightTopVal * a *b

In [15]:
def interpolationNear(image, x,y):
    a=x%1
    b = y%1
    if a >= 0.5:
        if b >= 0.5:
            
    b= int(y)+1 - y
    bilinearInterpolation = leftBottomVal * (1- a)*(1-b) + leftTopVal * (1-a) * b + rightBottomVal * a*(1-b) + rightTopVal * a *b

IndentationError: expected an indented block (5274181.py, line 7)

In [271]:
def pixelIntensity(rgb):
     return 0.299 * rgb[0] + 0.587 * rgb[1] + 0.114 * rgb[2]

In [272]:
def warp(image,homography):
    t_inverse = np.linalg.inv(homography)
    boundries = np.matmul(homography, np.array([[0,0,640,640],
                                                [0,480,0,480],
                                                [1,1,1,1]]))
    boundriesDenormalized = np.matmul(boundries, np.diag(1/boundries[-1]))
    print(boundriesDenormalized, np.shape(boundriesDenormalized))
    inversed = np.matmul(t_inverse, boundriesDenormalized)
    inversedNormalised = np.matmul(inversed, np.diag(1/inversed[-1]))
    print(inversedNormalised)
    newPositions = []
    xAxisRange = (int(np.amin(boundriesDenormalized[0])),int(np.amax(boundriesDenormalized[0])))
    minXIndex = np.where(boundriesDenormalized[0] == np.amin(boundriesDenormalized[0]))
    print('minx',minXIndex, boundriesDenormalized[0][minXIndex], boundriesDenormalized[1][minXIndex] )
    coordinates1 = np.matmul(t_inverse, np.array([[-539],[-136],[1]]))
    print("coordinates1",coordinates1 * (1/coordinates1[2][0]) )
    yAxisRange = (int(np.amin(boundriesDenormalized[1])),int(np.amax(boundriesDenormalized[1])))
    xAxisRange = (int(np.amin(boundriesDenormalized[0])),int(np.amax(boundriesDenormalized[0])))
    print(xAxisRange,yAxisRange)
    width = xAxisRange[1]-xAxisRange[0]
    height = yAxisRange[1]-yAxisRange[0]
    wrapped = np.zeros((height+1,width+1,3))
    print(height, width)
    print('traverseIndex',height-1,width-1)
    for h in range(height-1):
        for w in range(width-1):
            x = w + xAxisRange[0]
            y = h + yAxisRange[0]
            coordinates = np.matmul(t_inverse, np.array([[x],[y],[1]]))
            coordinates = (coordinates* (1/coordinates[2][0]))[0:2]
            if coordinates[0] > 0 and coordinates[1] > 0:
                if int(coordinates[1]) < len(image)-1 and int(coordinates[0]) < len(image[0])-1:
                    bilinearInterpolation  = interpolation(image,coordinates[1][0],coordinates[0][0])
                    wrapped[h][w] = bilinearInterpolation
    return np.asarray(wrapped), (xAxisRange[0],yAxisRange[0])

In [273]:
def mergeImages(warped, image, offsets):
    print(offsets)
    xOffset = offsets[0]
    yOffset = offsets[1]
    wWidth = np.shape(warped)[1]
    wHeight = np.shape(warped)[0]
    print(wWidth,wHeight)
    xLimit = wWidth + xOffset
    yLimit = wHeight + yOffset
    mergedX = max(len(image[0]), xLimit)
    mergedY = max(len(image), yLimit)
    merged = np.zeros(( mergedY - yOffset, mergedX- xOffset,3))
    print("warpedShape", np.shape(warped), 0 + xOffset, 0 + yOffset)
    print("mergedShape:",np.shape(merged))
    
    for i in range(len(warped)-1):
        for j in range(len(warped[0])-1):
            x = i + xOffset
            y = j + yOffset
            try:
                merged[i][j] = warped[i][j]
            except:
                print("i,j",i,j, "x,y",x,y)
            '''
            elif x < len(image[0]) and y < len(image):
                try:
                    merged[i][j] = image[i][j]
                except:
                    print("i,j",i,j, "x,y",x,y)
            else:
                merged[i][j] = warped[i][j]
            
            if x < len(image[0]) and y < len(image) and x > 0 and y > 0:
                try:
                    merged[i][j] = image[x][y]
                except e:
                    print(e)
                    print("i,j",i,j, "x,y",x,y)
            '''
            #elif x < len(image[0]) and y < len(image):           
    print(np.shape(merged),mergedX,mergedY)
    overlapped = warped[-yOffset:,-xOffset:]
    for i in range(len(image)-1):
        for j in range(len(image[1])-1):
            x = j - xOffset
            y = i - yOffset
            try:
                imageLuminance = pixelIntensity(image[i][j])
                warpedLuminance = pixelIntensity(merged[y][x])
                #print("i,j",i,j, "x,y",x,y)
                if imageLuminance >= warpedLuminance:
                    merged[y][x] = image[i][j]    
            except:
                print("i,j",i,j, "x,y",x,y) 
    print(np.shape(overlapped))
    #warped[-yOffset:,-xOffset:] = image[:yLimit,:xLimit]
    
    '''
    for i in range(len(overlapped)):
        for j in range(len(overlapped[0])):
    '''        
    return merged

In [274]:
def fwarp(image,homography):
    wrapped = np.zeros(np.shape(image))
    newPositions = []
    for i in range(len(image)):
        for j in range(len(image[i])):
            coordinates = np.matmul(homography, np.array([[i],[j],[1]]))
            newPositions.append(coordinates* (1/coordinates[2][0]))
    return np.asarray(newPositions).reshape(np.shape(np.asarray(image)))[:,:,0:2]

In [275]:
back_res,offset = warp(img2,homography)

[[ 3.08058201e+02  2.89178824e+02  1.05724555e+03  1.31463392e+03]
 [ 5.34177057e+01  4.73205769e+02 -1.21926316e+02  6.38662184e+02]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00]] (3, 4)
[[1.43690480e-13 1.19836733e-13 6.40000000e+02 6.40000000e+02]
 [6.58515346e-14 4.80000000e+02 5.47376300e-14 4.80000000e+02]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]]
minx (array([1]),) [289.17882405] [473.20576899]
coordinates1 [[-2.22057805e+05]
 [-9.49581144e+04]
 [ 1.00000000e+00]]
(289, 1314) (-121, 638)
759 1025
traverseIndex 758 1024


In [276]:
overlapped = mergeImages(back_res, img1, offset)
print("sonuc", np.shape(overlapped))
imgResult = Image.fromarray(overlapped.astype(np.uint8), 'RGB')
imgResult.save("overlapped.jpg")

(289, -121)
1026 760
warpedShape (760, 1026, 3) 289 -121
mergedShape: (760, 1026, 3)
(760, 1026, 3) 1315 639
(639, 289, 3)
sonuc (760, 1026, 3)


In [147]:
np.shape(overlapped)

(237, 627, 3)

In [121]:
np.shape(back_res)
 .299r' + .587g' + .114b'

(837, 977, 3)

In [93]:
back_res[1][0]

array([114.74235712, 140.90942738, 167.90942738])

In [277]:
imgResult = Image.fromarray(back_res.astype(np.uint8), 'RGB')
imgResult.save("warpedfirstImage5.jpg")

In [22]:
def construct(image,wrap):
    res = np.round(wrap)
    x_min = np.amin(res[:,:,0])
    y_min = np.amin(res[:,:,1])
    res[:,:,0] = res[:,:,0] + np.abs(x_min)
    res[:,:,1] = res[:,:,1] + np.abs(y_min)
    x_max = np.amax(res[:,:,0]).astype(int)
    y_max = np.amax(res[:,:,1]).astype(int)
    res = res.astype(int)
    print(x_max,y_max)
    scale_X = (len(image) / x_max) 
    scale_Y = (len(image[0]) / y_max)
    projected = np.zeros(np.shape(image))
    for i in range(len(image)):
        for j in range(len(image[i])):
            c = res[i][j]
            #print(int(c[0] * scale_X), int(c[1] * scale_Y) )
            projected[int(c[0] * scale_X)-1][int(c[1] * scale_Y)-1] = image[i][j]
    return projected

In [23]:
imageMatrix=construct(im,res)
imgResult = Image.fromarray(imageMatrix.astype(np.uint8), 'RGB')
imgResult.save("10transform.jpg")

3388 2739


In [24]:
print(np.amin(res[:,:,0]),np.amax(res[:,:,0]))
print(np.amin(res[:,:,1]),np.amax(res[:,:,1]))

-3138.7226735397476 248.77620614530588
-136.12788992140727 2602.759349287685


In [51]:
np.shape(np.array([[1],[2],[3]]))

(3, 1)

In [61]:
int(4.6)

4

In [140]:
3138.72267%1

0.7226700000001074